##Creating a datframe for the list of postal codes of Toronto


---



In [4]:
#importing the required libraries for the project
from geopy.geocoders import Nominatim
import requests
import pandas as pd
import numpy as np
from IPython.display import Image 
from IPython.core.display import HTML
from pandas.io.json import json_normalize
import folium
from sklearn.cluster import KMeans
import json
import matplotlib.cm as cm
import matplotlib.colors as colors 
from bs4 import BeautifulSoup

In [31]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
r = requests.get(URL) 
  
soup = BeautifulSoup(r.content, 'html5lib') 
table = soup.find('div', attrs = {'id':'container'})
print("WikiPage Scraped Successfully")

WikiPage Scraped Successfully


In [38]:
postalCodes= [];
boroughs= [];
neighborhoods = [];
columnNum = 1;
passVal = False

for row in soup.find_all('td'):
    for cell in row:
        if cell.string and cell.string[0].isalpha() and len(cell.string) > 2:         
            passVal = False
            if columnNum == 1:
                if passVal == False and cell.string[1].isdigit():
                    postalCodes.append(cell.string);   
                    columnNum = 2
                else:
                    continue
            elif columnNum == 2 :
                if cell.string == 'Not assigned':
                    passVal = True
                    del postalCodes[-1]
                    columnNum = 1
                    continue
                else:
                    boroughs.append(cell.string);      
                    columnNum = 3
            elif columnNum == 3 :
                if cell.string == 'Not assigned':
                    neighborhoods.append(boroughs[-1])
                else:
                    neighborhoods.append(cell.string); 
                columnNum = 1

print('Data Collected Successfully')

Data Collected Successfully


In [24]:
# define the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighborhood'] 

# instantiate the dataframe
neighbors = pd.DataFrame(columns=column_names)

neighbors


,PostalCode,Borough,Neighborhood


In [39]:
for data in range(len(neighborhoods)):
    code = postalCodes[data]
    borough = boroughs[data]
    neighborhood_name = neighborhoods[data]

    neighbors = neighbors.append({ 'PostalCode': code,
                                   'Borough': borough,
                                   'Neighborhood': neighborhood_name}, ignore_index=True)
neighbors.head()



,PostalCode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [42]:
neighbors["Borough"]=neighbors.Borough.str.replace('\\n','')
neighbors["PostalCode"]=neighbors.PostalCode.str.replace('\\n','')
neighbors["Neighborhood"]=neighbors.Neighborhood.str.replace('\\n','')

neighbors.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [58]:
neighbors.drop(neighbors[neighbors['Borough']=='Not assigned'].index,axis=0,inplace=True)
neighbors.drop(neighbors[neighbors['Neighborhood']=='Not assigned'].index,axis=0,inplace=True)
neighbors.reset_index(drop=True,inplace=True)

neighbors.head()


,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [70]:
neighbors = neighbors.groupby(['PostalCode','Borough'])['Neighborhood'].apply(', '.join).reset_index()
neighbors.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge, Malvern, Rouge, Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek, Rouge ..."
2,M1E,Scarborough,"Guildwood, Morningside, West Hill, Guildwood, ..."
3,M1G,Scarborough,"Woburn, Woburn, Woburn"
4,M1H,Scarborough,"Cedarbrae, Cedarbrae, Cedarbrae"


In [72]:
neighbors.to_csv('Toronto.csv',index=False)

In [73]:
neighbors.shape

(103, 3)